In [1]:
!pip -q install git+https://github.com/huggingface/transformers accelerate
!pip -q install qwen_vl_utils
!pip install bitsandbytes --upgrade --quiet
!pip install -q "outlines[transformers]"


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12=

In [6]:
from PIL import Image
import pandas as pd
from pydantic import BaseModel
import json
import re

import base64
import torch
#from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
from outlines import models as om
from outlines import generate as og
print("Libraries Imported")

Libraries Imported


In [3]:

# Directory containing the PDF files
img_folder = '/kaggle/input/illegible-medical-prescription-images-dataset/data'
output_excel = '/kaggle/working/qwen2-5-7b.xlsx'

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # Enable 4-bit quantization
    bnb_4bit_quant_type="nf4",    # Use Normal Float 4 (NF4) quantization
    bnb_4bit_use_double_quant=True,  # Double quantization for better efficiency
    bnb_4bit_compute_dtype=torch.float16  # Use float16 for computation
)


In [7]:
MODEL_ID = "Qwen/Qwen2.5-VL-7B-Instruct" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading multimodal model {MODEL_ID} on {DEVICE}...")
# ── Load & wrap the multimodal model with Outlines ────────────────────────
print(f"Loading {MODEL_ID} with Outlines vision wrapper …")
model = om.transformers_vision(
    MODEL_ID,
    model_class=Qwen2_5_VLForConditionalGeneration,  # correct HF class
    device=None,
    model_kwargs={           # forwarded to .from_pretrained(...)
        "device_map": "auto",
        "quantization_config": bnb_config,
        "torch_dtype": "auto",
    },
    tokenizer_class=AutoTokenizer,
)
print("✅ Model ready.")
processor = AutoProcessor.from_pretrained(MODEL_ID)
print("Multimodal model loaded successfully!")


Loading multimodal model Qwen/Qwen2.5-VL-7B-Instruct on cuda...
Loading Qwen/Qwen2.5-VL-7B-Instruct with Outlines vision wrapper …


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/57.6k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

✅ Model ready.
Multimodal model loaded successfully!


In [8]:
#from pydantic import BaseModel


class ClinicalRecord(BaseModel):
    patient_name: str
    diagnosis: str
    prescription: str


json_gen = og.json(model, ClinicalRecord)
print("Done")

Done


In [19]:
def generate_from_multimodal(prompt_text, images, max_tokens, temperature, top_p):
    """Generate text from multimodal inputs using Qwen2-VL"""
    record = json_gen(
        prompt_text,   # raw chat template with an [IMG] token
        images         # List[PIL.Image.Image]
    )

    return record
    


def create_chat_completion(messages_dict):
    prompt = processor.apply_chat_template(
        messages_dict, tokenize=False, add_generation_prompt=True
    )
    images, videos = process_vision_info(messages_dict)
    # Generate response using the prompt and images
    response_text = generate_from_multimodal(
        prompt_text=prompt,
        images=images, #processed_content.get("images", []),
        max_tokens=2048,
        temperature= 1,
        top_p=1
    )

    return response_text

print("Done")


def encode_image(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def check_and_resize_image(image, max_width=256, max_height=512):
    width, height = image.size
    # Check if the image resolution exceeds the max allowed
    if width > max_width or height > max_height:
        # Calculate the new dimensions maintaining the aspect ratio
        aspect_ratio = width / height
        if width > height:
            new_width = max_width
            new_height = int(new_width / aspect_ratio)
        else:
            new_height = max_height
            new_width = int(new_height * aspect_ratio)
        
        # Resize the image
        image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    
    return image

Done


In [21]:
variable_list = ["patient_name", "diagnosis", "prescription"] 


# Function to send multiple images in a single API call
def extract_structured_data_from_images(images):
    # Encode all images
    images_ = [check_and_resize_image(image) for image in images]
    base64_images = [encode_image(image) for image in images_]
    # Create message content with all the images embedded
    message_content = [
        
        {
            "type": "text",
            "text": f"Act as a Doctor and read images carefully. Extract the following clinical variables from the provided images and structure them according to these categories: {variable_list}. If any variable is not present in the image, return 'Not specified'." 
        }
    ]

    # Add each image to the message content
    for base64_image in base64_images:
        message_content.append({
            "type": "image",
            "image": f"data:image/jpeg;base64,{base64_image}"
        })
    
    message = [
            {
                "role": "system",
                "content": [
                    {"type": "text",
                     "text" : """Extract the text elements described by the user from the pictures, and return the result in structured JSON format : {variable_name : value}
                             Provide output for each variable strictly as it is mentioned. Strictly ensure that the variable names in output must be identicle to input variable list.     
                            Return only valid JSON with no extra explanations.
                            """
                    }
                ]
            },
            {
                "role": "user",
                "content": message_content
            },
        
        ]

    # Call the GPT-4o-mini model via OpenAI API
    structured_info = create_chat_completion(message)
    
    return structured_info

print("Done")

Done


In [22]:
import os
import time
import io
import gc
structured_data = []

# Loop through all image files in the folder
for idx, img_file in enumerate(sorted(os.listdir(img_folder)), 1):
    if not img_file.lower().endswith(('.jpg')):
        continue

    img_path = os.path.join(img_folder, img_file)
    print(f"[{idx}] Processing {img_file}…")
    start_time = time.time()

    # 1) Load the image
    image = Image.open(img_path)

    # 2) Run your GPT-based extractor
    torch.cuda.empty_cache()
    gc.collect()
    extracted_info = extract_structured_data_from_images([image])
    extraction_time = time.time() - start_time

    record = {
        'Image_ID': os.path.splitext(img_file)[0],
        'Extraction_Time_s': extraction_time,
        **extracted_info.dict() #(parsed if isinstance(parsed, dict) else {'parsed': parsed})
    }
    structured_data.append(record)

# 6) Save to Excel
df = pd.DataFrame(structured_data)
df.to_excel(output_excel, index=False)
print(f"Done! Results saved to {output_excel}")


[1] Processing 1.jpg…


/tmp/ipykernel_31/1127285340.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  **extracted_info.dict() #(parsed if isinstance(parsed, dict) else {'parsed': parsed})


[2] Processing 10.jpg…
[3] Processing 100.jpg…
[4] Processing 101.jpg…
[5] Processing 102.jpg…
[6] Processing 103.jpg…
[7] Processing 104.jpg…
[8] Processing 105.jpg…
[9] Processing 106.jpg…
[10] Processing 107.jpg…
[11] Processing 108.jpg…
[12] Processing 109.jpg…
[13] Processing 11.jpg…
[14] Processing 110.jpg…
[15] Processing 111.jpg…
[16] Processing 112.jpg…
[17] Processing 113.jpg…
[18] Processing 114.jpg…
[19] Processing 115.jpg…
[20] Processing 116.jpg…
[21] Processing 117.jpg…
[22] Processing 118.jpg…
[23] Processing 119.jpg…
[24] Processing 12.jpg…
[25] Processing 120.jpg…
[26] Processing 121.jpg…
[27] Processing 122.jpg…
[28] Processing 123.jpg…
[29] Processing 124.jpg…
[30] Processing 125.jpg…
[31] Processing 126.jpg…
[32] Processing 127.jpg…
[33] Processing 128.jpg…
[34] Processing 129.jpg…
[35] Processing 13.jpg…
[36] Processing 14.jpg…
[37] Processing 15.jpg…
[38] Processing 16.jpg…
[39] Processing 17.jpg…
[40] Processing 18.jpg…
[41] Processing 19.jpg…
[42] Processing 2.

In [23]:
df

,Image_ID,Extraction_Time_s,patient_name,diagnosis,prescription
0,1,5.488448,Patient 30,Not specified,Diprofen 0.125 mg tablet da no.7
1,10,5.003311,Mr. CR SAMUEL,shl vere sepiosis with MODS,Not specified
2,100,7.174057,Dr. W. F. Baker,Not specified,F.o. drambol. Lb. gr. 1/6. gr. 9/15-3 sty. cro...
3,101,2.943836,To change,Not specified,Not specified
4,102,5.384174,Not specified,"No (U)String seen, 1UU Holes","Ultraound to check positioning UC, RT after US..."
...,...,...,...,...,...
124,95,7.144327,BABY/SWLI DUBEY 01-2646-132005,Bilateral Profound Hearing Loss.,CT scan temporal bone & MRI - Cochlear Implant...
125,96,4.344285,Kevin Zionack,Not specified,Ferrrous Sulfate 4 mg PO TID; Dispense one bottle
126,97,9.441022,John Smith,Not specified,"Betaloc 100mg - 1 tab BID, Dorzolamidum 10 mg ..."
127,98,7.706424,Sh立场ndu Kumar,Not specified,Inj. RENDESIN 200mg in Day 1 Followed by Do an...
